<a href="https://colab.research.google.com/github/anko191/Python_Kaggle/blob/master/Feature_Engineering/Feature_Engineering_3_Encodings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Categorical Encoding か
* count encoding
* target encoding
* CatBoost encoding
を学びましょう

In [25]:
# とりあえず必要なもの

import pandas as pd
from sklearn.preprocessing import LabelEncoder

ks = pd.read_csv('/content/ks-projects-201801.csv',
                 parse_dates=['deadline', 'launched'])

# Drop live projects
ks = ks.query('state != "live"')

# Add outcome column, "successful" == 1, others are 0
ks = ks.assign(outcome=(ks['state'] == 'successful').astype(int))

# Timestamp features
ks = ks.assign(hour=ks.launched.dt.hour,
               day=ks.launched.dt.day,
               month=ks.launched.dt.month,
               year=ks.launched.dt.year)

# Label encoding
cat_features = ['category', 'currency', 'country']
encoder = LabelEncoder()
encoded = ks[cat_features].astype(str).apply(encoder.fit_transform)

data_cols = ['goal', 'hour', 'day', 'month', 'year', 'outcome']
data = ks[data_cols].join(encoded)

# Defining  functions that will help us test our encodings
import lightgbm as lgb
from sklearn import metrics

def get_data_splits(dataframe, valid_fraction=0.1):
    valid_fraction = 0.1
    valid_size = int(len(dataframe) * valid_fraction)

    train = dataframe[:-valid_size * 2]
    # valid size == test size, last two sections of the data
    valid = dataframe[-valid_size * 2:-valid_size]
    test = dataframe[-valid_size:]
    
    return train, valid, test

def train_model(train, valid):
    feature_cols = train.columns.drop('outcome')

    dtrain = lgb.Dataset(train[feature_cols], label=train['outcome'])
    dvalid = lgb.Dataset(valid[feature_cols], label=valid['outcome'])

    param = {'num_leaves': 64, 
             'objective': 'binary', 
             'metric': 'auc', 
             'seed': 7}
    bst = lgb.train(param, dtrain, num_boost_round=1000, valid_sets=[dvalid], 
                    early_stopping_rounds=10, verbose_eval=False)

    valid_pred = bst.predict(valid[feature_cols])
    valid_score = metrics.roc_auc_score(valid['outcome'], valid_pred)
    print(f"Validation AUC score: {valid_score:.4f}")

In [26]:
train, valid,test = get_data_splits(data)
train_model(train, valid)

Validation AUC score: 0.7467


## Count Encoding なんぞこれ
<b>データセット中で出現した回数で置き換えられる！！</b><br>
例えば、'GB'っていうのが10回でたら、'GB'は10という数字に置き換えられる！！

In [28]:
!pip install category_encoders

     |████████████████████████████████| 81kB 2.3MB/s 


In [29]:
import category_encoders as ce
cat_features = ['category', 'currency', 'country']

# Create the encoder
count_enc = ce.CountEncoder()

# _count を名前に追加するぞ
count_encoded = count_enc.fit_transform(ks[cat_features])
data = data.join(count_encoded.add_suffix("_count"))

# Train a model
train, valid, test = get_data_splits(data)
train_model(train, valid)

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Validation AUC score: 0.7486


ちょっとだけ上がってるｗ

## Target Encoding なにこれ？
<b>カテゴリ値を、その特徴の値に対するターゲットの平均値に置き換えます。</b><br>例えば、"CA"が与えられた場合、国 == "CA"のすべての行の平均結果を計算すると、約0.28になる。<br>
これは、出現頻度の少ない値の分散を減らすために、データセット全体のターゲット確率とブレンドされることが多いです。<br>
* ターゲットを利用して新しい特徴を作成
    * そのため、検証データやテストデータをターゲットエンコーディングに含めると、
    * **ターゲットリーク**となります。
    * その代わり、学習データセットからのみ、それを学習して、他に適用させろ

In [30]:
target_enc = ce.TargetEncoder(cols = cat_features)
target_enc.fit(train[cat_features], train['outcome'])

train_TE = train.join(target_enc.transform(train[cat_features]).add_suffix('_target'))
valid_TE = valid.join(target_enc.transform(valid[cat_features]).add_suffix('_target'))

train_model(train_TE, valid_TE)

Validation AUC score: 0.7491


またちょっとあがったね

## CatBoost Encoding なにこれ
* 与えられた値に対するターゲットの確率に基づいています

In [31]:
target_enc = ce.CatBoostEncoder(cols = cat_features)
target_enc.fit(train[cat_features], train['outcome'])

train_CBE = train.join(target_enc.transform(train[cat_features]).add_suffix('_cb'))
valid_CBE = valid.join(target_enc.transform(valid[cat_features]).add_suffix('_cb'))

train_model(train_CBE, valid_CBE)

Validation AUC score: 0.7492


はいはい

これらのエンコーディングは、カウントや平均などのデータセットから計算されていますね<br>
どのようなデータをしようすればいいのかな？検証データ使うのかな？テストデータ使えるかな？

* 学習データからのみやる必要があります。
    * 検証セットとテストセットのデータをエンコーディングに含めると、モデルの性能を過大評価することになります。
    

In [ ]:
import category_encoders as ce

cat_features = ['ip', 'app', 'device', 'os', 'channel']
train, valid, test = get_data_splits(clicks)

In [ ]:
# Create the count encoder
count_enc = ce.CountEncoder(cols = cat_features)

# Learn encoding from the training set
count_enc.fit(train[cat_features]) # いらないの

# Apply encoding to the train and validation sets as new columns
# Make sure to add `_count` as a suffix to the new columns
train_encoded = train.join(count_enc.transform(train[cat_features]).add_suffix('_count'))
valid_encoded = valid.join(count_enc.transform(valid[cat_features]).add_suffix('_count'))

# Check your answer
q_2.check()

In [ ]:
def get_data_splits(dataframe, valid_fraction=0.1):
    """Splits a dataframe into train, validation, and test sets.

    First, orders by the column 'click_time'. Set the size of the 
    validation and test sets with the valid_fraction keyword argument.
    """

    dataframe = dataframe.sort_values('click_time')
    valid_rows = int(len(dataframe) * valid_fraction)
    train = dataframe[:-valid_rows * 2]
    # valid size == test size, last two sections of the data
    valid = dataframe[-valid_rows * 2:-valid_rows]
    test = dataframe[-valid_rows:]
    
    return train, valid, test

def train_model(train, valid, test=None, feature_cols=None):
    if feature_cols is None:
        feature_cols = train.columns.drop(['click_time', 'attributed_time',
                                           'is_attributed'])
    dtrain = lgb.Dataset(train[feature_cols], label=train['is_attributed'])
    dvalid = lgb.Dataset(valid[feature_cols], label=valid['is_attributed'])
    
    param = {'num_leaves': 64, 'objective': 'binary', 
             'metric': 'auc', 'seed': 7}
    num_round = 1000
    bst = lgb.train(param, dtrain, num_round, valid_sets=[dvalid], 
                    early_stopping_rounds=20, verbose_eval=False)
    
    valid_pred = bst.predict(valid[feature_cols])
    valid_score = metrics.roc_auc_score(valid['is_attributed'], valid_pred)
    print(f"Validation AUC score: {valid_score}")
    
    if test is not None: 
        test_pred = bst.predict(test[feature_cols])
        test_score = metrics.roc_auc_score(test['is_attributed'], test_pred)
        return bst, valid_score, test_score
    else:
        return bst, valid_score

In [ ]:
# Train the model on the encoded datasets
# This can take around 30 seconds to complete
_ = train_model(train_encoded, valid_encoded)

### なんでカウンティんぐエンコーディングそんないいん？？
* 希少な値は、同じようなカウントを持つ傾向がある
* 予測時に希少な値をまとめて分類することが出来る。
* カウントが大きい共通の値は、他の値と正確なカウントが同じである可能性が低い
* そのために、共通の値 / 重要な値には独自のグループ訳がされるんよ

### Target encoding　するぞ

In [ ]:
# Create the target encoder. You can find this easily by using tab completion.
# Start typing ce. the press Tab to bring up a list of classes and functions.
target_enc = ce.TargetEncoder(cols = cat_features)

# Learn encoding from the training set. Use the 'is_attributed' column as the target.
target_enc.fit(train[cat_features], train['is_attributed'])

# Apply encoding to the train and validation sets as new columns
# Make sure to add `_target` as a suffix to the new columns
train_encoded = train.join(target_enc.transform(train[cat_features]).add_suffix('_target'))
valid_encoded = valid.join(target_enc.transform(valid[cat_features]).add_suffix('_target'))

# Check your answer
q_4.check()

In [ ]:
_ = train_model(train_encoded, valid_encoded)

#### IP Encodingを消してみる
符号化された特徴量から、 ip を除外し、ターゲット符号化でモデルを再学習すると、<br>スコアが増加し、ベースラインのスコアを上回ることに気付く
<br>IPアドレスをエンコードするとスコアがベースラインより低くなるが、
<br>エンコードしないとベースラインより高くなるのはなぜだと思いますか？

* ターゲット・エンコーディングは，カテゴリカル特徴量の各レベルのターゲットの母集団平均を測定しようとする．
* これは、レベルごとのデータが少ない場合、推定された平均は "真の "平均から遠く離れ、分散が多くなることを意味します。
* IPアドレスごとのデータが少ないので、推定値は他の特徴よりもはるかにノイズが多い可能性があります。モデルは非常に予測性が高いので、この特徴に大きく依存します。
* これにより、他の特徴での分割が少なくなり、それらの特徴はIPアドレスを考慮して残された誤差だけで適合されます。
* そのため、トレーニング・データにない新しいIPアドレスを見たとき（これはほとんどの新しいデータであると思われます）、モデルのパフォーマンスは非常に悪くなります。* 今後は、異なるエンコーディングを試す際にIP特徴を除外することにします。

### CatBoost Encoder
LightGBMモデルと相性が良いようですね

In [ ]:
# Remove IP from the encoded features
cat_features = ['app', 'device', 'os', 'channel']

# Create the CatBoost encoder
cb_enc = ce.CatBoostEncoder(cols=cat_features, random_state=7)

# Learn encoding from the training set
cb_enc.fit(train[cat_features], train['is_attributed'])

# Apply encoding to the train and validation sets as new columns
# Make sure to add `_cb` as a suffix to the new columns
train_encoded = train.join(cb_enc.transform(train[cat_features]).add_suffix('_cb'))
valid_encoded = valid.join(cb_enc.transform(valid[cat_features]).add_suffix('_cb'))

# Check your answer
q_6.check()

In [ ]:
_ = train_model(train_encoded, valid_encoded)